In [1]:
from keybert import KeyBERT
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import mysql.connector
from sqlalchemy import create_engine
from tqdm import tqdm 

In [2]:
#dataset = pd.read_csv('gxs_reviews.csv', index_col=0)

In [3]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_apple_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsapple = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_1744\3694027278.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gxsapple = pd.read_sql(query, con=cnx)


In [4]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM gxs_playstore_app_reviews"

# Use pandas to load sql query into a DataFrame
gxsplaystore = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_1744\1891013442.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gxsplaystore = pd.read_sql(query, con=cnx)


In [5]:
gxsapple['title_review'] = gxsapple['title'] + ' : ' + gxsapple['review']

gxsapple['thumbsUp']= 0

gxsapple['developerResponse'] = gxsapple['developerResponse'].apply(lambda x: x.get('body') if isinstance(x, dict) else x)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace(r"\{'id': \d+, 'body':", "", regex=True)
gxsapple['developerResponse']=gxsapple['developerResponse'].str.rstrip("}")
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace('"', '', regex=False)
gxsapple['developerResponse'] = gxsapple['developerResponse'].str.replace("'", '', regex=False)

gxsapple_dropped = gxsapple.drop(columns=['title', 'review', 'userName', 'isEdited'])

gxsplaystore_dropped=gxsplaystore.drop(columns=['reviewId','userName','userImage','reviewCreatedVersion','repliedAt','appVersion'])
neworder=['replyContent', 'score', 'content', 'thumbsUpCount', 'at']
gxsplaystore_dropped=gxsplaystore_dropped[neworder]

gxsplaystore_dropped=gxsplaystore_dropped.rename(columns={
    'content': 'content',
    'thumbsUpCount': 'thumbsUpCount',
    'replyContent': 'replyContent',
    'score': 'score',
    'at':'date'
})

gxsapple_dropped_renamed = gxsapple_dropped.rename(columns={
    'title_review': 'content',
    'thumbsUp': 'thumbsUpCount',
    'developerResponse': 'replyContent',
    'rating': 'score',
    'date':'date'
})

dataset = pd.concat([gxsplaystore_dropped, gxsapple_dropped_renamed], axis=0, ignore_index=True)
if dataset['replyContent'].apply(lambda x: isinstance(x, dict)).any():
    dataset['replyContent'] = dataset['replyContent'].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)


In [6]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date
0,Hi there! Thank you for your 5-star rating rev...,5,I like how the app is really light and fast co...,0,2024-04-15 02:17:49
1,"Hi Jerry, this feature isn't ready at the mome...",1,Cannot add card to google pay. Also cannot add...,0,2024-04-13 08:22:18
2,Thanks for your feedback on mobile wallets and...,4,Generally good. However please add either NFC ...,0,2024-04-13 05:53:34
3,Hi Eric. We're sorry to hear this. We would li...,1,"With continuous failed to log in,contacted cso...",0,2024-04-11 16:59:52
4,Hi Hafidz Melek! Thank you for your 5-star rat...,5,I don't get the negative reviews here. I insta...,0,2024-04-09 13:58:31


In [7]:
sa_model = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank'
sentiment_pipeline = pipeline("sentiment-analysis", model=sa_model)

In [8]:
ke_model = KeyBERT()

In [9]:
baseline_model = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(baseline_model)

save_path = "./gpt_model_causallm"
finetuned_model = AutoModelForCausalLM.from_pretrained(save_path)

generator = pipeline('text-generation', finetuned_model, tokenizer=tokenizer)
inst = 'Below is a review for a banking app. Write a response that appropriately replies to the review from the perspective of the bank.'

In [10]:
sentiment = []
keywords = []
replies = []

for index, row in dataset.iterrows():
    review = row['content']
    s = sentiment_pipeline(review)[0]['label']
    sentiment.append(s)
    k = ke_model.extract_keywords(review, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=1)
    if k == []:
        k = ''
    else: k = k[0][0]
    keywords.append(k)
    prompt = f'{inst} ### Review: {review} ### Reply:'
    reply = generator(prompt, max_length=len(review)+50)[0]['generated_text'][152+len(review):]
    replies.append(reply)

dataset['sentiment'] = sentiment
dataset['keywords'] = keywords
dataset['generatedReply'] = replies

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for 

In [11]:
dataset = dataset.rename(columns={'replies':'generatedReply'})

In [12]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date,sentiment,keywords,generatedReply
0,Hi there! Thank you for your 5-star rating rev...,5,I like how the app is really light and fast co...,0,2024-04-15 02:17:49,POSITIVE,bank apps,"Hi @JK, your review really helped us out a lo..."
1,"Hi Jerry, this feature isn't ready at the mome...",1,Cannot add card to google pay. Also cannot add...,0,2024-04-13 08:22:18,NEGATIVE,google pay,Hi John. My review is from 8th June 2010 and ...
2,Thanks for your feedback on mobile wallets and...,4,Generally good. However please add either NFC ...,0,2024-04-13 05:53:34,POSITIVE,sgqr codes,Thank you again for your feedback about the i...
3,Hi Eric. We're sorry to hear this. We would li...,1,"With continuous failed to log in,contacted cso...",0,2024-04-11 16:59:52,NEGATIVE,cso terminate,Hi I know where your device is and you don't ...
4,Hi Hafidz Melek! Thank you for your 5-star rat...,5,I don't get the negative reviews here. I insta...,0,2024-04-09 13:58:31,POSITIVE,negative reviews,I still really like it tho my friend :) 🙏🏉💜 💜...


In [13]:
dataset.keywords

0             bank apps
1            google pay
2            sgqr codes
3         cso terminate
4      negative reviews
             ...       
330                    
331            scan pay
332            bug 安全漏洞
333         singpass qr
334         gxs account
Name: keywords, Length: 335, dtype: object

In [14]:
#dataset2 = pd.read_csv('sentiment_data.csv', index_col=0)
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Construct the connection string using the loaded configuration
user = config['user']
password = config['password']
host = config['host']
dbname = config['database']
port = config['port']
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}'

# Create an engine
engine = create_engine(conn_string)

dataset.to_sql(name='sentiment_data', con=engine, if_exists='replace', index=False)


335

In [15]:
# dataset2['keywords'] = keywords

In [16]:
# dataset2

In [17]:
# dataset2.to_csv('sentiment_data.csv')

In [18]:
gxs_reviews = dataset[['content', 'thumbsUpCount', 'sentiment', 'keywords']]

In [19]:
def summarise_sentiment(reviews): #havent test this fn

  '''
  Input:
  - reviews: dataset containing at least the following 4 columns ['content', 'thumbsUpCount', 'sentimment', 'keywords']
  Output:
  - net sentiment score: numeric, between -100 and +100
  - neg_keywords: dictionary containing negative keywords, sorted descending
  - pos_keywords: dictionary containing positive keywords, sorted descending
  '''

  ns_score = 0
  neg_keywords = {}
  pos_keywords = {}

  for index, row in reviews.iterrows():
    pred_sentiment = row['sentiment']
    # keywords = row['keywords']
    word = row['keywords']
    if pred_sentiment == 'POSITIVE':
      pred_sentiment = 1
      # for word in keywords:
      if word in pos_keywords:
        pos_keywords[word] += 1
      else:
        pos_keywords[word] = 1
    else:
      pred_sentiment = -1
      # for word in keywords:
      if word in neg_keywords:
        neg_keywords[word] += 1
      else:
        neg_keywords[word] = 1
    ns_score += pred_sentiment

  total = len(reviews)
  ns_score = ns_score/total*100

  if '' in pos_keywords: del pos_keywords['']
  if '' in neg_keywords: del neg_keywords['']
  pos_keywords = dict(sorted(pos_keywords.items(), key=lambda x: x[1], reverse = True))
  neg_keywords = dict(sorted(neg_keywords.items(), key=lambda x: x[1], reverse = True))

  return ns_score, pos_keywords, neg_keywords

In [20]:
summary = summarise_sentiment(gxs_reviews)

In [33]:
summary1=summary

In [34]:
summary1

(-17.611940298507463,
 {'easy use': 4,
  'awesome app': 4,
  'good': 3,
  'ease use': 3,
  'banking app': 3,
  'great app': 3,
  'good app': 2,
  'gxs bank': 2,
  'uiux app': 2,
  'gxs': 2,
  'saving pocket': 2,
  'digital banking': 2,
  'bank app': 2,
  'app design': 2,
  'bank apps': 1,
  'sgqr codes': 1,
  'negative reviews': 1,
  'gif mony': 1,
  'gxs simple': 1,
  'multiple pockets': 1,
  'fast repayments': 1,
  'ok good': 1,
  'using bank': 1,
  'savings pockets': 1,
  'load apps': 1,
  'fast app': 1,
  'easy loan': 1,
  'digital savings': 1,
  'great savings': 1,
  'loving far': 1,
  'rebates app': 1,
  'activate gxs': 1,
  'secure transactions': 1,
  'use payments': 1,
  'great bank': 1,
  'super efficient': 1,
  'superb': 1,
  'transfer bank': 1,
  'friendly user': 1,
  'loan foreigner': 1,
  'month gxs': 1,
  'easy fingertips': 1,
  'loan loan': 1,
  'cool app': 1,
  'seamless': 1,
  'love gxs': 1,
  'setup pocket': 1,
  'payee feature': 1,
  'apps easy': 1,
  'seamless regis

In [43]:
#summary_df = pd.DataFrame(summary, index=['netSentiment', 'positiveKeywords', 'negativeKeywords'])
positive_df = pd.DataFrame(list(summary1[1].items()), columns=['keyword', 'count'])
negative_df = pd.DataFrame(list(summary1[2].items()), columns=['keyword', 'count'])
positive_df['netSentiment'] = summary1[0]  # Add netSentiment as a column to positive_df
negative_df['netSentiment'] = summary1[0]  # Add netSentiment as a column to negative_df

positive_df['sentiment'] = 'positive'
negative_df['sentiment'] = 'negative'

summary_df = pd.concat([positive_df, negative_df], ignore_index=True)

print(summary_df)

          keyword  count  netSentiment sentiment
0        easy use      4     -17.61194  positive
1     awesome app      4     -17.61194  positive
2            good      3     -17.61194  positive
3        ease use      3     -17.61194  positive
4     banking app      3     -17.61194  positive
..            ...    ...           ...       ...
287  use maribank      1     -17.61194  negative
288         pr sp      1     -17.61194  negative
289      scan pay      1     -17.61194  negative
290      bug 安全漏洞      1     -17.61194  negative
291   gxs account      1     -17.61194  negative

[292 rows x 4 columns]


In [46]:
#summary_df.to_csv('summary.csv')

#dataset.to_csv('sentiment_data.csv')
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Construct the connection string using the loaded configuration
user = config['user']
password = config['password']
host = config['host']
dbname = config['database']
port = config['port']
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}'

# Create an engine
engine = create_engine(conn_string)

summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)


292